### Season 4 Phenotype Table
#### Started November 2019 after GenoPhenoEnvo Kickoff
#### Columns
* `range`
* `column`
* `canopy_height`
* `max_canopy_height`
* `days_to_` multiple values

In [2]:
%pwd
%cd '/Users/ejcain/Dev/UA-AG/phenotypes/terraref-datasets/'

/Users/ejcain/Dev/UA-AG/phenotypes/terraref-datasets


In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

* Season 4 data queried from betydb using R, then downloaded as csv
* Some columns have already been dropped in the very messy season_4_transformation notebook

In [3]:
df = pd.read_csv('more_traits_after_feedback.csv')
print(df.shape)
df.head()

(145623, 14)


,site_id,id,sitename,lat,lon,cultivar_id,date,time,raw_date,trait,trait_description,mean,units,cultivar
0,6000005593,6002011659,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,26.505859,%,PI511355
1,6000005593,6002011662,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,43.840000,C,PI511355
2,6000005593,6002011666,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,Difference between the leaf temperature and th...,-2.800000,C,PI511355
3,6000005361,6002012219,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,20.522461,%,PI570373
4,6000005361,6002012222,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,44.189999,C,PI570373


In [17]:
# df.trait.astype('object').combine(df['mean'], func=)    

In [16]:
# new_df = df.assign(ambient_humidity=" ")


for index, row in new_df.iterrows():
    if row['trait'] == 'ambient_humidity':
        new_df.loc[index, ['ambient_humidity']] = row['mean']
        
new_df.head()b

,site_id,id,sitename,lat,lon,cultivar_id,date,time,raw_date,trait,trait_description,mean,units,cultivar,ambient_humidity
0,6000005593,6002011659,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,26.505859,%,PI511355,26.5059
1,6000005593,6002011662,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,43.840000,C,PI511355,
2,6000005593,6002011666,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,Difference between the leaf temperature and th...,-2.800000,C,PI511355,
3,6000005361,6002012219,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,20.522461,%,PI570373,20.5225
4,6000005361,6002012222,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,44.189999,C,PI570373,


In [19]:
# this works
    
for index, row in new_df.iterrows():
    if row['trait'] == 'ambient_humidity':
        new_df.loc[index, ['ambient_humidity']] = row['mean']
        
new_df.head()

,site_id,id,sitename,lat,lon,cultivar_id,date,time,raw_date,trait,trait_description,mean,units,cultivar,ambient_humidity
0,6000005593,6002011659,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,26.505859,%,PI511355,26.5059
1,6000005593,6002011662,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,43.840000,C,PI511355,
2,6000005593,6002011666,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,Difference between the leaf temperature and th...,-2.800000,C,PI511355,
3,6000005361,6002012219,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,20.522461,%,PI570373,20.5225
4,6000005361,6002012222,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,44.189999,C,PI570373,


In [23]:
# will not scale, but will work for now

# trait_columns = ['proximal_air_temperature', 'leaf_temperature_differential', 'surface_temperature',
#                 'aboveground_dry_biomass', 'seedling_emergence_rate', 'canopy_height', 
#                 'flag_leaf_emergence_rate', 'flowering_time', 'grain_stage_time', 'canopy_cover',
#                 'emergence_count']

# for trait in trait_columns:
    
# for index, row in new_df.iterrows():
#     if row['trait'] == 'aboveground_dry_biomass':
#         new_df.loc[index, ['aboveground_dry_biomass']] = row['mean']
        
# new_df.head()

In [6]:
df.trait.unique()

array(['ambient_humidity', 'proximal_air_temperature',
       'leaf_temperature_differential', 'surface_temperature',
       'aboveground_dry_biomass', 'seedling_emergence_rate',
       'canopy_height', 'flag_leaf_emergence_time', 'flowering_time',
       'grain_stage_time', 'canopy_cover', 'emergence_count'],
      dtype=object)

#### New Trait Columns

In [24]:
new_df.trait.unique()

array(['ambient_humidity', 'proximal_air_temperature',
       'leaf_temperature_differential', 'surface_temperature',
       'aboveground_dry_biomass', 'seedling_emergence_rate',
       'canopy_height', 'flag_leaf_emergence_time', 'flowering_time',
       'grain_stage_time', 'canopy_cover', 'emergence_count'],
      dtype=object)

Change column name `mean` due to being a keyword and also confusing

In [8]:
df.rename({'mean': 'value'}, axis=1, inplace=True)
df.tail()

,site_id,id,sitename,lat,lon,cultivar_id,date,time,raw_date,trait,trait_description,value,units,cultivar
145618,6000005865,6004797282,MAC Field Scanner Season 4 Range 41 Column 10,33.075985,-111.974901,6000000969,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",392.0,cm,PI573193
145619,6000005866,6004797283,MAC Field Scanner Season 4 Range 41 Column 11,33.075985,-111.974884,6000000688,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",386.0,cm,PI152860
145620,6000005882,6004797287,MAC Field Scanner Season 4 Range 41 Column 15,33.075985,-111.974819,6000000953,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",342.0,cm,PI570087
145621,6000005906,6004797288,MAC Field Scanner Season 4 Range 42 Column 2,33.076021,-111.975032,6000000751,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",331.0,cm,PI257600
145622,6000005888,6004797290,MAC Field Scanner Season 4 Range 42 Column 4,33.076021,-111.974999,6000000225,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",370.0,cm,PI154844


Drop `trait_description`

In [9]:
df.drop(labels='trait_description', axis=1, inplace=True)

. . . and units

In [12]:
df.drop(labels='units', axis=1, inplace=True)

In [13]:
df.shape

(145623, 12)

In [16]:
df.isnull().sum()

site_id        0
id             0
sitename       0
lat            0
lon            0
cultivar_id    0
date           0
time           0
raw_date       0
trait          0
value          0
cultivar       0
dtype: int64

* Set sitename as index
* 
*

In [18]:
sitename_index = df.set_index(keys='sitename')
print(sitename_index.shape)
sitename_index.head()

(145623, 11)


,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar
sitename,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,PI511355
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,PI511355
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,-2.800000,PI511355
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012219,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,20.522461,PI570373
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012222,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,44.189999,PI570373


In [19]:
sitename_index.dtypes

site_id          int64
id               int64
lat            float64
lon            float64
cultivar_id      int64
date            object
time            object
raw_date        object
trait           object
value          float64
cultivar        object
dtype: object

#### Pivot Table / Group By / Stack / Crosstab ?!?!? 

In [64]:
sitename_index.groupby('trait').value.mean()

trait
aboveground_dry_biomass          21655.360000
ambient_humidity                    28.810093
canopy_cover                        22.666667
canopy_height                      157.584539
emergence_count                     50.219081
flag_leaf_emergence_time            64.556962
flowering_time                      67.685185
grain_stage_time                    76.229167
leaf_temperature_differential       -4.161549
proximal_air_temperature            42.811384
seedling_emergence_rate              0.771159
surface_temperature                 36.712652
Name: value, dtype: float64

In [68]:
# this
# does
# not
# work
# traits_df = sitename_index.unstack('trait', fill_value=' ')
# traits_df.head()

* Create pivot table with traits, then concat back with the initial df
* Should be the same length as original df if NA rows aren't dropped

In [94]:
# this also doesn't work

# trait_df = sitename_index.pivot(index=sitename_index.index, columns='trait',
#                          values='value')

# print(trait_df.shape)
# trait_df.head()

In [95]:
sample_df = sitename_index.sample(n=50, random_state=42)
sample_df.head()

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,range_location,column_location
sitename,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 34 Column 7,6000005813,6002275455,33.075734,-111.974950,6000000928,2017 Jul 12,12:00:00 (America/Phoenix),2017-07-12 14:00:00 -0500,surface_temperature,37.711786,PI569444,34,7
MAC Field Scanner Season 4 Range 14 Column 2,6000005642,6002226190,33.075015,-111.975031,6000000819,2017 Jun 29,12:00:00 (America/Phoenix),2017-06-29 14:00:00 -0500,surface_temperature,35.260431,PI330182,14,2
MAC Field Scanner Season 4 Range 35 Column 16,6000005803,6002274171,33.075770,-111.974803,6000001054,2017 Jul 9,12:00:00 (America/Phoenix),2017-07-09 14:00:00 -0500,surface_temperature,39.858606,SP1615,35,16
MAC Field Scanner Season 4 Range 17 Column 11,6000005745,6002236468,33.075123,-111.974884,6000000732,2017 Jul 31,12:00:00 (America/Phoenix),2017-07-31 14:00:00 -0500,surface_temperature,37.235498,PI196049,17,11
MAC Field Scanner Season 4 Range 44 Column 11,6000005913,6002238275,33.076093,-111.974885,6000000842,2017 Aug 10,12:00:00 (America/Phoenix),2017-08-10 14:00:00 -0500,surface_temperature,35.545251,PI505717,44,11


In [96]:
sample_df.trait.unique()

array(['surface_temperature', 'canopy_height'], dtype=object)

In [ ]:
# Code from Rafa 

# from collections import defaultdict
# ​
# # make each trait a key, so you can make it a column later
# traits = df.trait.unique
# ​
# # create dictionary
# trait_columns_dict = {}
# ​
# # set all the traits as empty lists
# for trait in traits:
# 	trait_columns_dict[trait] = []
	
# # this is inefficient but it will work
# for trait, value in zip(df.trait, df.value):
# 	for col in traits:
# 		if trait == col:
# 			trait_columns_dict.append(value)
# 		else:
# 			trait_columns_dict.append(None)
# ​
# trait_columns_dict
# 	df

#### Add `column` and `range` columns for easier location identification

In [69]:
sitename_index.head()

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar
sitename,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,PI511355
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,PI511355
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,-2.800000,PI511355
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012219,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,20.522461,PI570373
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012222,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,44.189999,PI570373


In [70]:
sitename_values = sitename_index.index.values
print(len(sitename_values))
print(sitename_values[:10])

145623
['MAC Field Scanner Season 4 Range 26 Column 15'
 'MAC Field Scanner Season 4 Range 26 Column 15'
 'MAC Field Scanner Season 4 Range 26 Column 15'
 'MAC Field Scanner Season 4 Range 21 Column 2'
 'MAC Field Scanner Season 4 Range 21 Column 2'
 'MAC Field Scanner Season 4 Range 21 Column 2'
 'MAC Field Scanner Season 4 Range 21 Column 2'
 'MAC Field Scanner Season 4 Range 21 Column 2'
 'MAC Field Scanner Season 4 Range 21 Column 2'
 'MAC Field Scanner Season 4 Range 21 Column 2']


In [71]:
no_e_w_names = []

for name in sitename_values:
    
    if name.endswith(' W') | name.endswith(' E'):
        name = name[:-2]
        no_e_w_names.append(name)
        
    else:
        no_e_w_names.append(name)

print(len(no_e_w_names))
print(no_e_w_names[:10])

145623
['MAC Field Scanner Season 4 Range 26 Column 15', 'MAC Field Scanner Season 4 Range 26 Column 15', 'MAC Field Scanner Season 4 Range 26 Column 15', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2']


In [72]:
for name in no_e_w_names:
    
    if name.endswith(' W'):
        print(name)
        
    if name.endswith(' E'):
        print(name)
        
    if name.endswith(' '):
        print('This is wrong too.')

In [73]:
test_sitename_string = no_e_w_names[42]
test_sitename_string

'MAC Field Scanner Season 4 Range 26 Column 14'

In [76]:
split_string = test_sitename_string.split(' ')

In [77]:
print(split_string[6])
print(split_string[8])

26
14


In [79]:
ranges = []
columns = []

for name in no_e_w_names:
    
    split_name = name.split(' ')
    
    range_value = split_name[6]
    ranges.append(range_value)
    
    column_value = split_name[8]
    columns.append(column_value)
    
print(f'There are {len(ranges)} range values!')
print(f'First three range values: {ranges[:3]}')

print(f'There are {len(columns)} column values!')
print(f'Here are the first three column values! {columns[:3]}')

print(f'All of these numbers should be {len(sitename_index)}')

There are 145623 range values!
First three range values: ['26', '26', '26']
There are 145623 column values!
Here are the first three column values! ['15', '15', '15']
All of these numbers should be 145623


In [80]:
sitename_index.head(3)

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar
sitename,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,PI511355
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,PI511355
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,-2.800000,PI511355


In [81]:
sitename_index['range_location'] = ranges
sitename_index['column_location'] = columns

sitename_index.tail()

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,range_location,column_location
sitename,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 41 Column 10,6000005865,6004797282,33.075985,-111.974901,6000000969,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,392.0,PI573193,41,10
MAC Field Scanner Season 4 Range 41 Column 11,6000005866,6004797283,33.075985,-111.974884,6000000688,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,386.0,PI152860,41,11
MAC Field Scanner Season 4 Range 41 Column 15,6000005882,6004797287,33.075985,-111.974819,6000000953,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,342.0,PI570087,41,15
MAC Field Scanner Season 4 Range 42 Column 2,6000005906,6004797288,33.076021,-111.975032,6000000751,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,331.0,PI257600,42,2
MAC Field Scanner Season 4 Range 42 Column 4,6000005888,6004797290,33.076021,-111.974999,6000000225,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,370.0,PI154844,42,4


In [84]:
# check that `E` and `W` columns have accurate range and column values

sitename_index.loc[sitename_index.index.str.endswith(' E')].sample(n=7)

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,range_location,column_location
sitename,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 19 Column 9 E,6000007536,6001940812,33.075195,-111.974913,6000000839,2017 May 29 (America/Phoenix),[time unspecified or unknown],2017-05-30 00:00:00 -0500,canopy_height,61.000,PI455280,19,9
MAC Field Scanner Season 4 Range 37 Column 14 E,6000008122,6001945957,33.075842,-111.974831,6000000914,2017 Jul 2 (America/Phoenix),[time unspecified or unknown],2017-07-03 00:00:00 -0500,canopy_height,265.000,PI569244,37,14
MAC Field Scanner Season 4 Range 34 Column 3 E,6000008004,6001941220,33.075734,-111.975011,6000000977,2017 May 29 (America/Phoenix),[time unspecified or unknown],2017-05-30 00:00:00 -0500,canopy_height,45.000,PI585461,34,3
MAC Field Scanner Season 4 Range 37 Column 8 E,6000008110,6001941314,33.075842,-111.974929,6000000880,2017 May 29 (America/Phoenix),[time unspecified or unknown],2017-05-30 00:00:00 -0500,canopy_height,60.000,PI562970,37,8
MAC Field Scanner Season 4 Range 36 Column 2 E,6000008066,6001944147,33.075806,-111.975028,6000000707,2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,131.000,PI155885,36,2
MAC Field Scanner Season 4 Range 18 Column 9 E,6000007504,6001950708,33.075159,-111.974913,6000000213,2017 May 2 (America/Phoenix),[time unspecified or unknown],2017-05-03 00:00:00 -0500,seedling_emergence_rate,0.968,PI535785,18,9
MAC Field Scanner Season 4 Range 20 Column 2 E,6000007554,6001946132,33.075230,-111.975027,6000001061,2017 Jul 11 (America/Phoenix),[time unspecified or unknown],2017-07-12 00:00:00 -0500,canopy_height,110.000,PI542718,20,2


In [85]:
# check W's

sitename_index.loc[sitename_index.index.str.endswith(' W')].sample(n=4)

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,range_location,column_location
sitename,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 41 Column 8 W,6000008237,6001944818,33.075985,-111.974938,6000000912,2017 May 24 (America/Phoenix),[time unspecified or unknown],2017-05-25 00:00:00 -0500,canopy_height,49.0,PI569097,41,8
MAC Field Scanner Season 4 Range 42 Column 7 W,6000008267,6001944332,33.076021,-111.974954,6000000560,2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,121.0,PI329319,42,7
MAC Field Scanner Season 4 Range 17 Column 4 W,6000007461,6001950693,33.075123,-111.975003,6000000954,2017 May 2 (America/Phoenix),[time unspecified or unknown],2017-05-03 00:00:00 -0500,emergence_count,51.0,PI570090,17,4
MAC Field Scanner Season 4 Range 14 Column 15 W,6000007387,6001940685,33.075015,-111.974823,6000001003,2017 May 29 (America/Phoenix),[time unspecified or unknown],2017-05-30 00:00:00 -0500,canopy_height,62.0,PI641850,14,15


In [86]:
sitename_index.to_csv('column_and_range_df.csv')